 <h1 align="center">Operation Atlas: Decrypting Insights from the CIA Factbook</h1>

<center>
    <iframe src="https://giphy.com/embed/Hz1hFjTarLPJGyOUNI" width="480" height="270" frameborder="0"></iframe>
</center>

<h3 align="center"> by Thais Lovisi</h3>

The current project aims to work with data from the CIA World Factbook, to extract insights into demographic and geographic information. 

The Factbook is a compendium of statistics about all the recognized countries on Earth. Yep! You are right, it doesn't include data from Minas Gerais, which was claimed by the Greys in the 90s when they land at Varginha.

Jokes aside, the Factbook is a fantastic data collection with infinite possibilities to analyze. Some of the gathered data is:

- population — the global population.
- population_growth — the annual population growth rate, as a percentage.
- area — the total land and water area.

All the data used at this project was extracted at: https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db

## Load the SQL extension and File

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0




In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook(1).db

## Terrain reconnaissance : Knowing my table

As good agents, we know that before taking on any mission, thorough preparation is the sucess key. And in the world of data, the terrain reconnaissance begins with knowing our data.

Let's start by a quick overview.

In [3]:
%%sql
SELECT * 
    FROM sqlite_master
    WHERE type = 'table';

 * sqlite:///factbook(1).db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


This query retrieves information about tables in an SQLite database. The sqlite_master table is a system table in SQLite that stores metadata about the database schema, including information about tables, views, and other database objects.

By the output above, we have that this dataset has two tables named **sqlite_sequence** and **facts**. The table **sqlite_sequence** contains the column's name and seq.  **facts** contains:
- id type integer and is a PRIMARY KEY
- code type varchar(255)
- name type varchar(255)
- area type integer
- area_land type integer
- area_water type integer
- population type integer
- population_growt type float
- birth_rate type float
- death_rate type float
- migration_rate type float


Time to start to see things closer to find escape routes and problems that need to be dealt with before the real mission starts. We can start by taking a look at the first 5 rows.

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook(1).db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

name — the name of the country.
<br>area— the country's total area (is the sum of land and water area).
<br>area_land — the country's land area in square kilometers.
<br>area_water — the country's water area in square kilometers.
<br>population — the country's population.
<br>population_growth— the country's population growth as a percentage.
<br>birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
<br>death_rate — the country's death rate, or the number of death per year per 1,000 people.

### Summary statistics and Outliers search

Yet on the terrain reconnaissance is necessary to spot the sneakiest enemies. 

In [5]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

 * sqlite:///factbook(1).db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In the summary statistics we can note that:
1. There's a country with a population of 0
2. There's a country with a population of more than 7.2 billion people

We can dig deeper on the info using subqueries as follows :

#### Finding the Country with 0 population

In [6]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MIN(population)
              FROM facts)

 * sqlite:///factbook(1).db
Done.


name,population
Antarctica,0


#### Finding the Country with 7.2million population

In [7]:
%%sql
SELECT name, population
    FROM facts
    ORDER BY population DESC
    LIMIT 1;

 * sqlite:///factbook(1).db
Done.


name,population
World,7256490011


Here it is! Our outlier is actually the total for all observation as the the World population is the sum of population of all countries for the year of 2015 according to [Worldometer](https://www.worldometers.info/world-population/world-population-by-year/). So in this case it is necessary to remove our outlier.

###### Remember: 
After a target neutralization is necessary to assess the situation again. The removal of a target can generate more trouble in the long run. Thus is necessary to make the reconnaissance again without them. So after removing a outlier, ***always*** do the summary again!

#### Summary statistics without Outliers

In [8]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook(1).db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [9]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook(1).db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


**ALL GOOD... PHASE 1 COMPLETED!** 🏅

## Operation ´Eagle Out of The Neast´ Starts

### Identifying Densely Populated Countries

In [10]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);

 * sqlite:///factbook(1).db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


#### Which country has the most people? Which country has the highest growth rate?

In [11]:
%%sql
SELECT name, population, birth_rate
    FROM facts
    WHERE population == (SELECT MAX(population)
                        FROM facts
                        WHERE name <> "World") OR birth_rate == (SELECT MAX(birth_rate)
                                                                 FROM facts
                                                                 WHERE name <> "World")

 * sqlite:///factbook(1).db
Done.


name,population,birth_rate
China,1367485388,12.49
Niger,18045729,45.45


From the data collected is possible to conclude that China has the biggest population. Yet, the country with the biggest growth rate expressed by birth_rate is Niger. For verify the top 5 countries in terms of biggest population, we can do ?

In [12]:
%%sql
SELECT name, population, birth_rate
    FROM facts
    WHERE name != "World"
    ORDER BY population DESC
    LIMIT 5;

 * sqlite:///factbook(1).db
Done.


name,population,birth_rate
China,1367485388,12.49
India,1251695584,19.55
European Union,513949445,10.2
United States,321368864,12.49
Indonesia,255993674,16.72


To identify the top 5 in number of births we can run the following query:

In [13]:
%%sql
SELECT name, birth_rate, death_rate
    FROM facts
    WHERE name != "World"
    ORDER BY birth_rate DESC
    LIMIT 5;

 * sqlite:///factbook(1).db
Done.


name,birth_rate,death_rate
Niger,45.45,12.42
Mali,44.99,12.89
Uganda,43.79,10.69
Zambia,42.13,12.67
Burkina Faso,42.03,11.72


#### What are the Countries with the smallest total population? 

In [14]:
%%sql
SELECT name, population, birth_rate
    FROM facts
    WHERE population != "None"
    ORDER BY population ASC
    LIMIT 5;

 * sqlite:///factbook(1).db
Done.


name,population,birth_rate
Antarctica,0,None
Pitcairn Islands,48,None
Cocos (Keeling) Islands,596,None
Holy See (Vatican City),842,None
Niue,1190,None


#### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [15]:
%%sql
SELECT name, cast(area_water as Float)/cast(area_land as Float) AS ratio_water_land
    FROM facts
    WHERE name <> 'World'
    ORDER BY ratio_water_land DESC
    LIMIT 1;
    

 * sqlite:///factbook(1).db
Done.


name,ratio_water_land
British Indian Ocean Territory,905.6666666666666


In [16]:
%%sql
SELECT name, area_water
    FROM facts
    WHERE area_water > area_land
    ORDER BY area_water DESC

 * sqlite:///factbook(1).db
Done.


name,area_water
British Indian Ocean Territory,54340
Virgin Islands,1564


British Indian Ocean Territory is easier to get away by boat, there is water everywhere the ratio_water land is the biggest one followed by Virgin Islands.

#### Which countries will add the most people to their populations next year?

In [17]:
%%sql
SELECT name, MAX(population_growth - migration_rate) As increase
    FROM facts

 * sqlite:///factbook(1).db
Done.


name,increase
Malawi,3.32


In [18]:
%%sql
SELECT name, (population_growth - migration_rate) As increase
    FROM facts
    WHERE increase > (SELECT AVG(population_growth - migration_rate) As increase
                        FROM facts
                      WHERE name <> 'World') AND increase > 1.0
    ORDER BY increase DESC

 * sqlite:///factbook(1).db
Done.


name,increase
Malawi,3.32
Burundi,3.28
Burkina Faso,3.03
Gaza Strip,2.81
Benin,2.78
Niger,2.69
Togo,2.69
Ethiopia,2.67
Guinea,2.63
Madagascar,2.58


If we consider solely the increase it will be Malawi.

#### Which countries have a higher death rate than birth rate?

In [19]:
%%sql
SELECT name, death_rate, birth_rate
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate DESC

 * sqlite:///factbook(1).db
Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


The East European countries Ukraine (14.46), Bulgaria (14.44), and Latvia (14.3) are at the top 3 on the list. Countries that were part of Soviet Union experienced a drastic decline on birth rate since 1989. This can be explained by a social and economic dynamic change that follows after the fall of URSS as Sobotka, Tomáš(2019). 

In Ukraine and Latvia, problems related to healthcare administration still be cause of death by infectious diseases in Latvia, Ukraine, and Bulgaria. See information as [here.](https://worldpopulationreview.com/country-rankings/death-rate-by-country)

#### Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [20]:
%%sql
SELECT name, cast(population as Float)/cast(area as Float) AS ratio_pop_area
    FROM facts
    WHERE ratio_pop_area > (SELECT AVG(cast(population as Float)/cast(area as Float)) AS avg_ratio_pop_area
                            FROM facts
                            WHERE name <> 'World')
    ORDER BY ratio_pop_area DESC


 * sqlite:///factbook(1).db
Done.


name,ratio_pop_area
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


In [21]:
%%sql
SELECT name, death_rate, birth_rate,cast(population as Float)/cast(area as Float) AS ratio_pop_area
    FROM facts
    WHERE death_rate > birth_rate AND ratio_pop_area > (SELECT AVG(cast(population as Float)/cast(area as Float)) AS avg_ratio_pop_area
                            FROM facts
                            WHERE name <> 'World')  

 * sqlite:///factbook(1).db
Done.


name,death_rate,birth_rate,ratio_pop_area
Monaco,9.24,6.65,15267.5


When merging queries about higher death rate than birth rate and which countries have the highest population/area ratio, was possible tp notice that only Monaco is in both queries.

### Additional Query just because I'm curious about Brazil, Poland and Italy

In [22]:
%%sql
SELECT *
    FROM facts
    WHERE name IN ('Brazil', 'Poland', 'Italy')

 * sqlite:///factbook(1).db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46


From the table facts is possible to notice that the Brazil has shown a biggest population Growth but smaller migration_rate.

In [23]:
%%sql
SELECT name, area, cast(population as Float)/cast(area as Float) AS ratio_pop_area, cast(area_water as Float)/cast(area_land as Float) AS ratio_water_land,
        cast(birth_rate as Float)/cast(death_rate as Float) AS ratio_birth_death
    FROM facts
   WHERE name IN ('Brazil', 'Poland', 'Italy')

 * sqlite:///factbook(1).db
Done.


name,area,ratio_pop_area,ratio_water_land,ratio_birth_death
Brazil,8515770,23.98606491250938,0.018859459161966657,2.197568389057751
Italy,301340,205.26687462666754,0.024478139661385734,0.8577036310107949
Poland,312685,123.3259958104802,0.027707022070302872,0.955839057899902


For the variable ratio_pop_area Italy has a bigger number of inhabitants per km2. Some possible explanations are:
 - Italy is a thin peninsula, where the inland area is much smaller. 
 - Throughout history, Italian coastal areas were more populated due to trade and fishing. The South Italy culture was strongly influenced by the sea. It contributed to the inland occupation structure, once it was made to ease the circulation of people and products from the coast to land.  This influenced the demography of the country. 
 - A strong cultural sense of community that makes even in rural areas people agglomerate. 
 - A high migration.    
 
About the ratio_birth_death, Brazil has the bigger rate. This could be explained by the effective public child health care provided by SUS. The SUS handles preventive and curative care with access to the whole country.

An interesting fact is that the values from death_rate for Poland dropped after a serie of restructuration at the public health care ZUS that is taking place since 2000s. The most recent change was in [2017](https://www.sciencedirect.com/science/article/pii/S0168851018306237).

In [24]:
%%sql
SELECT AVG(area), AVG(cast(population as Float)/cast(area as Float)) AS avg_ratio_pop_area, AVG(cast(area_water as Float)/cast(area_land as Float)) AS avg_ratio_water_land,
AVG(cast(birth_rate as Float)/cast(death_rate as Float)) AS ratio_birth_death
    FROM facts
    WHERE name <> 'World'

 * sqlite:///factbook(1).db
Done.


AVG(area),avg_ratio_pop_area,avg_ratio_water_land,ratio_birth_death
555093.546184739,419.66252469247945,3.803454624004138,2.8398621495152736


## Conclusion

With CIA World factbook data we proceed the identification of the potential grounds for operation. Our analysis is based on demographic indicators like population, mortality, and fertility.

	• Accounting for 46% of the world's total population, China, India, the United States, Indonesia, and Brazil are the most populated countries. The least populated world locations are majorly Islands and dependencies.

So is easier to go undercover at China, India, the United States, Indonesia, and Brazil if you are an adult. However Niger has bigger potential for our Operation Crib-Crawl. Of course it would demmand extra health budget due a high death_rate.

	• Due to their small land area, Poland and Italy are more densely populated than Brazil. 
	• Fertility is highest in Africa and lowest in Europe and Asia. Some African countries (Lesotho, Guinea-Bissau) and Eastern European countries (Bulgaria, Ukraine and Latvia) record the highest death rates. The Gulf cooperation countries of the Middle-East record the lowest death rates. If we where to analyse death/brith relation we could observe that the biggest values where held by  Eastern European countries who at the analised period had the low birth number and high child mortality.  
    
    • Population is rapidly declining in East and South-East Europe. This occurrence is due to reducing birth rates, increasing deaths, and emigration to other countries. Japan is also experiencing a population decline despite the positive forecasts for the country.
    
Operation Crib-Crawl isn't feasable on those, a spy baby would be easily spotted. Better send the The Seasoned Spy.

### Limitation

	•This data colllection is outdated. As a result, they may not reflect the current state of the world's countries.
    •Some countries present some data blackout, that can occur by unnintentional failure at data register, political or other intentional fake data registers, lack of official data transparence and problems of social orders that may incur in a error of estimation of demographical parameters. 

##### Additional Bibliography:

Sobotka, Tomáš. “Fertility in Central and Eastern Europe after 1989: Collapse and Gradual Recovery.” Historical Social Research / Historische Sozialforschung, vol. 36, no. 2 (136), 2011, pp. 246–96. JSTOR, http://www.jstor.org/stable/41151282. Accessed 8 June 2023.